# Kernel Density Estimation

This notebook is an introduction into the practical usage of KDEs in zfit and explains the different parameters.
*A complete introduction to Kernel Density Estimations, explanations to all methods implemented in zfit and a throughout
comparison of the performance can be either found in
[Performance of univariate kernel density estimation methods in TensorFlow](https://astroviking.github.io/ba-thesis/)
by Marc Steiner from which parts here are taken or in the [documentation of zfit](https://zfit.readthedocs.io/en/latest/)*


Kernel Density Estimation is a non-parametric method to estimate the density of a population and offers a more accurate way than a
histogram.
In a kernel density estimation each data point is substituted with a kernel function
that specifies how much it influences its neighboring regions. This kernel functions can then be summed up to get an
estimate of the probability density distribution, quite similarly as summing up data points inside bins.

However, since
the kernel functions are centered on the data points directly, KDE circumvents the problem of arbitrary bin positioning.
KDE still depends on the kernel bandwidth (a measure of the spread of the kernel function), however, the total PDF
does depend less strongly on the kernel bandwidth than histograms do on bin width and it is much easier to specify
rules for an approximately optimal kernel bandwidth than it is to do so for bin width.

## Definition

Given a set of $n$ sample points $x_k$ ($k = 1,\cdots,n$), kernel density estimation $\widehat{f}_h(x)$ is defined as

\begin{equation}
\widehat{f}_h(x) = \frac{1}{nh} \sum_{k=1}^n K\Big(\frac{x-x_k}{h}\Big)
(\#eq:kde)
\end{equation}

where $K(x)$ is called the kernel function, $h$ is the bandwidth of the kernel and $x$ is the value for which the estimate is calculated. The kernel function defines the shape and size of influence of a single data point over the estimation, whereas the bandwidth defines the range of influence. Most typically a simple Gaussian distribution ($K(x) :=\frac{1}{\sqrt{2\pi}}e^{-\frac{1}{2}x^2}$) is used as kernel function.
The larger the bandwidth parameter $h$ the larger is the range of influence of a single data point on the estimated distribution.

## Computational complexity

This leads for a large n however to computational problems, as the computational complexity of the exact KDE above is given by $\mathcal{O}(nm)$ where $n$ is the number of sample points to estimate from and $m$ is the number of evaluation points (the points where you want to calculate the estimate).

To circumvent this problem, there exist several approximative methods to decrease this complexity and therefore decrease the runtime as well.

**Hint on running the notebook**

Feel free to rerun a cell a few times. This will change the sample drawn and gives an impression of _how the PDF based on this sample could also look like_.

In [ ]:
import matplotlib.pyplot as plt
import mplhep
import numpy as np
import zfit

## Exact KDE

Using the definition above of a KDE, this is implemented in the `KDE1DimExact`. We can start out with a simple Gaussian.

In [ ]:
obs_wide = zfit.Space('x', (-10, 10))
x = np.linspace(-10, 10, 200)

In [ ]:
true_pdf = zfit.pdf.Gauss(obs=obs_wide, mu=0, sigma=2)
sample = true_pdf.sample(60)
sample_np = zfit.run(sample.value())

kde = zfit.pdf.KDE1DimExact(sample,
                            # obs,
                            # kernel,
                            # padding,
                            # weights,
                            # name
                            )
plt.plot(x, kde.pdf(x), label='Default exact KDE')
plt.plot(x, true_pdf.pdf(x), label='True PDF')
plt.plot(sample_np, np.zeros_like(sample_np), 'b|', ms=12)
plt.legend()

This looks already reasonable and we see that the PDF is overestimated in the regions where we sampled, by chance, many events and underestimated in other regions. Since this was a simple example, /et's create a more complitated one (and let's use a bit more samples in order to be able to infer the shape).

In [ ]:
gauss1 = zfit.pdf.Gauss(obs=obs_wide, mu=0, sigma=2)
gauss2 = zfit.pdf.Gauss(obs=obs_wide, mu=3, sigma=0.5)
true_pdf = zfit.pdf.SumPDF([gauss1, gauss2], fracs=0.85)
sample = true_pdf.sample(1000)
sample_np = zfit.run(sample.value())

kde = zfit.pdf.KDE1DimExact(sample,
                            # obs,
                            # kernel,
                            # padding,
                            # weights,
                            # name
                            )
plt.plot(x, kde.pdf(x), label='Default exact KDE')
plt.plot(x, true_pdf.pdf(x), label='True PDF')
plt.legend()

This is more difficult, actually impossible for the current configuration, to approximate the actual PDF well, because we use by default a single bandwidth.

## Bandwidth

The bandwidth of a kernel defines it's width, corresponding to the `sigma` of a Gaussian distribution. There is a distinction between global and local bandwidth:

<dl>
  <dt><strong>Global bandwidth</strong></dt>
  <dd>A is a single parameter that is shared amongst all kernels.
      While this is a fast and robust method,
      it is a rule of thumb approximation. Due to its global nature,
      it cannot take into account the different varying
      local densities.</dd>
  <dt><strong>Local bandwidth</strong></dt>
  <dd>A local bandwidth
      means that each kernel $i$ has a different bandwidth.
      In other words, given some data points with size $n$,
      we will need $n$ bandwidth parameters.
      This is often more accurate than a global bandwidth,
      as it allows to have larger bandwiths in areas of smaller density,
      where, due to the small local sample size, we have less certainty
      over the true density while having a smaller bandwidth in denser
      populated areas.</dd>

</dl>

We can compare the effects of different global bandwidths

In [ ]:
plt.plot(x, true_pdf.pdf(x), label='True PDF')

for h in [0.1, 0.5, 2, 10]:
    kde = zfit.pdf.KDE1DimExact(sample, bandwidth=h)
    plt.plot(x, kde.pdf(x), '--', alpha=0.6, label=f'KDE h={h}')
plt.legend()

Making the bandwidth larger makes the KDE less dependent on the randomness of the sample and overfitting of it while it tends to smear out features.
0.1 is clearly too wigly while 2 already smears out the feature of having actually two Gaussian peaks.

There are a few methods to estimate the optimal global bandwidth, Silvermans (default) and Scotts rule of thumb, respectively. There are also adaptive methods implemented that create an initial density estimation with a rule of thumb and use it to define local bandwidth that are inversely proportional to the (squareroot of) the density.

In [ ]:
kdes_all = [("silverman", '--'), ("scott", '--'), ("adaptive_geom", ':'), ("adaptive_zfit", ':'), ("isj", '-.')]
kdes_some = [("silverman", '--'), ("adaptive_zfit", ':'), ("isj", '-.')]
for subplot in range(1, 4):
    plt.figure(subplot)
    if subplot != 1:
        # let's zoom in to better see the details
        plt.xlim(-2.5, 4)
        plt.ylim(0.1, 0.2)
    plt.plot(x, true_pdf.pdf(x), 'k', label='True PDF')
    kdes = kdes_some if subplot == 3 else kdes_all
    for h, fmt in kdes:
        kde = zfit.pdf.KDE1DimExact(sample, bandwidth=h)
        plt.plot(x, kde.pdf(x), fmt, alpha=0.8, label=f'KDE {h}')
    plt.legend()

As we can see, the adaptive method better takes into account the nuances of the peaks. It is very well possible to use local bandwidths directly as an array parameter to bandwidth.

## Kernel

The kernel is the heart of the Kernel Density Estimation, which consists of the sum of
kernels around each sample point. Therefore, a kernel should represent
the distribution probability of a single data point as close as
possible.

The most widespread kernel is a Gaussian, or Normal, distribution as many real world example follow it.
However, there are many cases where this assumption is not per-se true. In
this cases an alternative kernel may offers a better choice.

Valid choices are callables that return a
[tensorflow_probability.distribution.Distribution](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions?version=nightly), such as all distributions
that belong to the [loc-scale family](https://en.wikipedia.org/wiki/Location%E2%80%93scale_family).


In [ ]:
import tensorflow_probability as tfp

tfd = tfp.distributions

In [ ]:
plt.plot(x, true_pdf.pdf(x), label='True PDF')

for kernel in [tfd.Normal, tfd.Cauchy, tfd.Moyal]:
    kde = zfit.pdf.KDE1DimExact(sample, kernel=kernel)
    plt.plot(x, kde.pdf(x), '--', alpha=0.6, label=f'KDE {kernel.__name__}')
plt.legend()

In [ ]:
## Boundary bias